# Artificial Neural Network

#### Importing the Libraries

In [1]:
import Pkg
Pkg.add("DataFrames")
Pkg.add("Flux")
Pkg.add("CSV")
Pkg.add("MLJ")
Pkg.add("CUDA")
Pkg.add("IterTools")
Pkg.add("ProgressMeter")

    Updating registry at `C:\Users\steve\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\steve\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes t

In [2]:
using DataFrames
using CSV
using Flux
using MLJ
using CUDA
using IterTools: ncycle

## Part 1 - Data Preprocessing

#### Importing the dataset

In [3]:
dataset = DataFrame(CSV.File("../data/Churn_Modelling.csv"))
first(dataset, 10)

Row,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String31,Int64,String7,String7,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0
6,6,15574012,Chu,645,Spain,Male,44,8,1.13756e5,2,1,0,1.49757e5,1
7,7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0
8,8,15656148,Obinna,376,Germany,Female,29,4,1.15047e5,4,1,0,1.19347e5,1
9,9,15792365,He,501,France,Male,44,4,1.42051e5,2,0,1,74940.5,0


In [4]:
cols(name) = name != (:CustomerId) && name != (:RowNumber) && name != (:Surname)

y, X = unpack(dataset, ==(:Exited), cols; rng=43)

([0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 1, 0, 0, 0, 0, 0, 1, 0], 10000×10 DataFrame
   Row │ CreditScore  Geography  Gender   Age    Tenure  Balance         NumOf ⋯
       │ Int64        String7    String7  Int64  Int64   Float64         Int64 ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │         602  Germany    Female      42       6       1.58415e5        ⋯
     2 │         792  France     Female      42       2       0.0
     3 │         633  Spain      Male        58       6   98308.5
     4 │         778  France     Male        44       8       1.23864e5
     5 │         619  France     Female      49       9  145360.0              ⋯
     6 │         840  Germany    Female      51       1   87779.8
     7 │         495  France     Male        47      10       1.37683e5
     8 │         747  France     Male        41       5       0.0
     9 │         633  Spain      Male        27       3       0.0              ⋯
    10 │         614 

In [5]:
first(X, 10) |> pretty

┌─────────────┬───────────┬─────────┬───────┬────────┬────────────┬───────────────┬───────────┬────────────────┬─────────────────┐
│ CreditScore │ Geography │ Gender  │ Age   │ Tenure │ Balance    │ NumOfProducts │ HasCrCard │ IsActiveMember │ EstimatedSalary │
│ Int64       │ String7   │ String7 │ Int64 │ Int64  │ Float64    │ Int64         │ Int64     │ Int64          │ Float64         │
│ Count       │ Textual   │ Textual │ Count │ Count  │ Continuous │ Count         │ Count     │ Count          │ Continuous      │
├─────────────┼───────────┼─────────┼───────┼────────┼────────────┼───────────────┼───────────┼────────────────┼─────────────────┤
│ 602         │ Germany   │ Female  │ 42    │ 6      │ 1.58415e5  │ 1             │ 1         │ 1              │ 1.31886e5       │
│ 792         │ France    │ Female  │ 42    │ 2      │ 0.0        │ 2             │ 1         │ 0              │ 92664.1         │
│ 633         │ Spain     │ Male    │ 58    │ 6      │ 98308.5    │ 1             │

In [6]:
first(y, 10)

10-element Vector{Int64}:
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0

### Encoding categorical data

In [7]:
schema(X)

┌─────────────────┬────────────┬─────────┐
│ names           │ scitypes   │ types   │
├─────────────────┼────────────┼─────────┤
│ CreditScore     │ Count      │ Int64   │
│ Geography       │ Textual    │ String7 │
│ Gender          │ Textual    │ String7 │
│ Age             │ Count      │ Int64   │
│ Tenure          │ Count      │ Int64   │
│ Balance         │ Continuous │ Float64 │
│ NumOfProducts   │ Count      │ Int64   │
│ HasCrCard       │ Count      │ Int64   │
│ IsActiveMember  │ Count      │ Int64   │
│ EstimatedSalary │ Continuous │ Float64 │
└─────────────────┴────────────┴─────────┘


#### Label Encoding the "Gender" column

In [8]:
X.Gender = coerce(X.Gender, Multiclass)
schema(X)

┌─────────────────┬───────────────┬───────────────────────────────────┐
│ names           │ scitypes      │ types                             │
├─────────────────┼───────────────┼───────────────────────────────────┤
│ CreditScore     │ Count         │ Int64                             │
│ Geography       │ Textual       │ String7                           │
│ Gender          │ Multiclass{2} │ CategoricalValue{String7, UInt32} │
│ Age             │ Count         │ Int64                             │
│ Tenure          │ Count         │ Int64                             │
│ Balance         │ Continuous    │ Float64                           │
│ NumOfProducts   │ Count         │ Int64                             │
│ HasCrCard       │ Count         │ Int64                             │
│ IsActiveMember  │ Count         │ Int64                             │
│ EstimatedSalary │ Continuous    │ Float64                           │
└─────────────────┴───────────────┴─────────────────────────────

In [9]:
levels(X.Gender)

2-element Vector{String7}:
 "Female"
 "Male"

In [10]:
first(X, 10)

Row,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
,Int64,String7,Cat…,Int64,Int64,Float64,Int64,Int64,Int64,Float64
1,602,Germany,Female,42,6,1.58415e5,1,1,1,1.31886e5
2,792,France,Female,42,2,0.0,2,1,0,92664.1
3,633,Spain,Male,58,6,98308.5,1,1,1,1.32034e5
4,778,France,Male,44,8,1.23864e5,1,1,0,1.44495e5
5,619,France,Female,49,9,145360.0,1,1,0,38186.8
6,840,Germany,Female,51,1,87779.8,1,0,1,36687.1
7,495,France,Male,47,10,1.37683e5,1,1,0,71071.5
8,747,France,Male,41,5,0.0,2,1,1,22750.2
9,633,Spain,Male,27,3,0.0,2,1,0,44008.9


#### One Hot Encoding the "Geography" column

In [11]:
X.Geography = coerce(X.Geography, Multiclass)

hot_encoder = OneHotEncoder(features=[:Geography, :Gender])
mach = fit!(machine(hot_encoder, X))
X = MLJ.transform(mach, X)
schema(X)

[ Info: Training machine(OneHotEncoder(features = [:Geography, :Gender], …), …).
[ Info: Spawning 3 sub-features to one-hot encode feature :Geography.
[ Info: Spawning 2 sub-features to one-hot encode feature :Gender.


┌────────────────────┬────────────┬─────────┐
│ names              │ scitypes   │ types   │
├────────────────────┼────────────┼─────────┤
│ CreditScore        │ Count      │ Int64   │
│ Geography__France  │ Continuous │ Float64 │
│ Geography__Germany │ Continuous │ Float64 │
│ Geography__Spain   │ Continuous │ Float64 │
│ Gender__Female     │ Continuous │ Float64 │
│ Gender__Male       │ Continuous │ Float64 │
│ Age                │ Count      │ Int64   │
│ Tenure             │ Count      │ Int64   │
│ Balance            │ Continuous │ Float64 │
│ NumOfProducts      │ Count      │ Int64   │
│ HasCrCard          │ Count      │ Int64   │
│ IsActiveMember     │ Count      │ Int64   │
│ EstimatedSalary    │ Continuous │ Float64 │
└────────────────────┴────────────┴─────────┘


In [12]:
first(X, 10)

Row,CreditScore,Geography__France,Geography__Germany,Geography__Spain,Gender__Female,Gender__Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Float64
1,602,0.0,1.0,0.0,1.0,0.0,42,6,1.58415e5,1,1,1,1.31886e5
2,792,1.0,0.0,0.0,1.0,0.0,42,2,0.0,2,1,0,92664.1
3,633,0.0,0.0,1.0,0.0,1.0,58,6,98308.5,1,1,1,1.32034e5
4,778,1.0,0.0,0.0,0.0,1.0,44,8,1.23864e5,1,1,0,1.44495e5
5,619,1.0,0.0,0.0,1.0,0.0,49,9,145360.0,1,1,0,38186.8
6,840,0.0,1.0,0.0,1.0,0.0,51,1,87779.8,1,0,1,36687.1
7,495,1.0,0.0,0.0,0.0,1.0,47,10,1.37683e5,1,1,0,71071.5
8,747,1.0,0.0,0.0,0.0,1.0,41,5,0.0,2,1,1,22750.2
9,633,0.0,0.0,1.0,0.0,1.0,27,3,0.0,2,1,0,44008.9


In [13]:
X = coerce(X,
    :Geography__France => OrderedFactor,
    :Geography__Germany => OrderedFactor,
    :Geography__Spain => OrderedFactor,
    :Gender__Female => OrderedFactor,
    :Gender__Male => OrderedFactor,
    :HasCrCard => OrderedFactor,
    :IsActiveMember => OrderedFactor)

first(X, 10) |> pretty

┌─────────────┬───────────────────────────────────┬───────────────────────────────────┬───────────────────────────────────┬───────────────────────────────────┬───────────────────────────────────┬───────┬────────┬────────────┬───────────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────┐
│ CreditScore │ Geography__France                 │ Geography__Germany                │ Geography__Spain                  │ Gender__Female                    │ Gender__Male                      │ Age   │ Tenure │ Balance    │ NumOfProducts │ HasCrCard                       │ IsActiveMember                  │ EstimatedSalary │
│ Int64       │ CategoricalValue{Float64, UInt32} │ CategoricalValue{Float64, UInt32} │ CategoricalValue{Float64, UInt32} │ CategoricalValue{Float64, UInt32} │ CategoricalValue{Float64, UInt32} │ Int64 │ Int64  │ Float64    │ Int64         │ CategoricalValue{Int64, UInt32} │ CategoricalValue{Int64, UInt32} │ Float64         │
│ Count       │ 

In [14]:
#y = coerce(y, OrderedFactor)
#y = convert(Vector{Int64}, y)


In [15]:
scitype(y)

AbstractVector{Count} (alias for AbstractArray{Count, 1})

In [16]:
scitype(X)

Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{OrderedFactor{2}}}}

### Feature Scaling

In [17]:
standardizer_model = Standardizer(count=true)
standardizer_machine = machine(standardizer_model, X)
fit!(standardizer_machine)

X = MLJ.transform(standardizer_machine, X)

[ Info: Training machine(Standardizer(features = Symbol[], …), …).


Row,CreditScore,Geography__France,Geography__Germany,Geography__Spain,Gender__Female,Gender__Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
,Float64,Cat…,Cat…,Cat…,Cat…,Cat…,Float64,Float64,Float64,Float64,Cat…,Cat…,Float64
1,-0.502092,0.0,1.0,0.0,1.0,0.0,0.293503,0.341335,1.31302,-0.911538,1,1,0.552877
2,1.4637,1.0,0.0,0.0,1.0,0.0,0.293503,-1.04171,-1.22579,0.807696,1,0,-0.129127
3,-0.181357,0.0,0.0,1.0,0.0,1.0,1.81908,0.341335,0.349736,-0.911538,1,1,0.555445
4,1.31885,1.0,0.0,0.0,0.0,1.0,0.4842,1.03286,0.75929,-0.911538,1,0,0.772115
5,-0.326205,1.0,0.0,0.0,1.0,0.0,0.960945,1.37862,1.1038,-0.911538,1,0,-1.07638
6,1.96032,0.0,1.0,0.0,1.0,0.0,1.15164,-1.38747,0.181,-0.911538,0,1,-1.10246
7,-1.60914,1.0,0.0,0.0,0.0,1.0,0.770247,1.72438,0.980759,-0.911538,1,0,-0.504582
8,0.998116,1.0,0.0,0.0,0.0,1.0,0.198154,-0.00442574,-1.22579,0.807696,1,1,-1.3448
9,-0.181357,0.0,0.0,1.0,0.0,1.0,-1.13673,-0.695947,-1.22579,0.807696,1,0,-0.97515


### Splitting the dataset into the Training Set and the Test set

In [18]:
(X_train, X_test), (y_train, y_test) = partition((X, y), 0.8, rng=43, multi=true)

((8000×13 DataFrame
  Row │ CreditScore  Geography__France  Geography__Germany  Geography__Spain   ⋯
      │ Float64      CategoricalValue…  CategoricalValue…   CategoricalValue…  ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │  -0.636593   1.0                0.0                 0.0                ⋯
    2 │   0.667036   0.0                0.0                 1.0
    3 │   0.460111   1.0                0.0                 0.0
    4 │  -0.36759    1.0                0.0                 0.0
    5 │  -2.22992    0.0                1.0                 0.0                ⋯
    6 │  -0.191704   1.0                0.0                 0.0
    7 │  -1.68156    1.0                0.0                 0.0
    8 │   0.12903    1.0                0.0                 0.0
    9 │  -0.843518   0.0                1.0                 0.0                ⋯
   10 │   0.884307   0.0                1.0                 0.0
   11 │  -1.43325    1.0                0.0   

## Part 2 - Building the ANN

#### Adding the input layer and the first hidden layer

In [19]:
layer1 = Dense(13 => 6, relu)

input_shape = size(X_train)
print(input_shape)
layer1(transpose(Matrix(X_train))) |> size

(8000, 13)

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(13 => 6, relu)  # 84 parameters
│   summary(x) = "13×8000 transpose(::Matrix{Float64}) with eltype Float64"
└ @ Flux C:\Users\steve\.julia\packages\Flux\uCLgc\src\layers\stateless.jl:50


(6, 8000)

#### Adding the second hidden layer

In [20]:
layer2 = Dense(6 => 6, relu)

Dense(6 => 6, relu)  # 42 parameters

#### Adding the output layer

In [21]:
output = Dense(6 => 1)

Dense(6 => 1)       # 7 parameters

#### Put it together


In [22]:
model = Chain(layer1, layer2, output)

Chain(
  Dense(13 => 6, relu),                 # 84 parameters
  Dense(6 => 6, relu),                  # 42 parameters
  Dense(6 => 1),                        # 7 parameters
)                   # Total: 6 arrays, 133 parameters, 916 bytes.

#### Test the model before training

In [23]:
output1 = model(transpose(Matrix(X_train)))

mean((output1[1, :] .> 0.5) .== y_train)

0.79875

In [24]:
size(output1)

(1, 8000)

## Part 3 - Training the ANN

#### Load the data into a dataloader and set batchsize

In [25]:
loader = Flux.DataLoader((transpose(Matrix(X_train)), y_train), batchsize=32, shuffle=true)

250-element DataLoader(::Tuple{LinearAlgebra.Transpose{Float64, Matrix{Float64}}, Vector{Int64}}, shuffle=true, batchsize=32)
  with first element:
  (13×32 Matrix{Float64}, 32-element Vector{Int64},)

#### Setup Optimizer

In [26]:
optimizer = Flux.setup(Flux.Adam(0.01), model)

(layers = ((weight = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999))), σ = ()), (weight = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999))), σ = ()), (weight = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam{Float64}(0.01, (0

In [27]:
loss(m, x, y) = Flux.logitbinarycrossentropy(transpose(m(x)), y)

loss (generic function with 1 method)

#### Training the ANN on the Training set

In [28]:
number_epochs = 10

for epoch in 1:number_epochs
    Flux.train!(loss, model, loader, optimizer)
end

optimizer

(layers = ((weight = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[-0.000253863 -0.00160249 … -0.000284484 -0.00185444; -0.00589272 9.15043f-5 … -0.0030733 -0.00649132; … ; 0.00227992 -0.00180385 … 0.000186809 -0.000585455; -0.00091484 0.00152096 … -0.00290088 -0.00377456], Float32[0.0015654 0.000651124 … 0.000705587 0.00159205; 0.00100588 0.000490875 … 0.00073384 0.000969348; … ; 0.00172569 0.000729717 … 0.000797649 0.00171054; 0.000745192 0.000346567 … 0.000497527 0.000651607], (3.6351e-115, 0.0819004))), bias = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.000304521, -0.000861429, 0.00220994, 0.00031512, 0.0012779, -0.00159603], Float32[0.00186134, 0.0011086, 0.0018676, 0.000118385, 0.00200784, 0.000780116], (3.6351e-115, 0.0819004))), σ = ()), (weight = Leaf(Adam{Float64}(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.000418912 -0.0200032 … -0.000546503 -0.000778983; 0.00258837 -0.000759717 … 0.00274983 -0.000701728; … ; 0.0116277 -0.00439686 … 0.0121247 -0.0040667

In [29]:
output2 = model(transpose(Matrix(X_train)))

mean((output2[1, :] .> 0.5) .== y_train)

0.863875

## Part 4 - Making the predictions and evaluating the model

#### Homework

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40
- Tenure: 3 years
- Balance: $60000
- Number of Products: 2

- Have a credit card?: Yes

- Active Member?: Yes

- Estimated Salary: $500000


In [30]:
names(X)

13-element Vector{String}:
 "CreditScore"
 "Geography__France"
 "Geography__Germany"
 "Geography__Spain"
 "Gender__Female"
 "Gender__Male"
 "Age"
 "Tenure"
 "Balance"
 "NumOfProducts"
 "HasCrCard"
 "IsActiveMember"
 "EstimatedSalary"

In [31]:
single_data = DataFrame(CreditScore = [600],
    Geography__France = [1],
    Geography__Germany = [0],
    Geography__Spain = [0],
    Gender__Female = [0],
    Gender__Male = [1],
    Age = [40], 
    Tenure = [3], 
    Balance = [60000],
    NumOfProducts = [2],
    HasCrCard = [1],
    IsActiveMember = [1],
    EstimatedSalary = [50000])

single_data = coerce(single_data,
    :Geography__France => OrderedFactor,
    :Geography__Germany => OrderedFactor,
    :Geography__Spain => OrderedFactor,
    :Gender__Female => OrderedFactor,
    :Gender__Male => OrderedFactor,
    :HasCrCard => OrderedFactor,
    :IsActiveMember => OrderedFactor)

Row,CreditScore,Geography__France,Geography__Germany,Geography__Spain,Gender__Female,Gender__Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
,Int64,Cat…,Cat…,Cat…,Cat…,Cat…,Int64,Int64,Int64,Int64,Cat…,Cat…,Int64
1,600,1,0,0,0,1,40,3,60000,2,1,1,50000


In [32]:
standard_data = MLJ.transform(standardizer_machine, single_data)

Row,CreditScore,Geography__France,Geography__Germany,Geography__Spain,Gender__Female,Gender__Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
,Float64,Cat…,Cat…,Cat…,Cat…,Cat…,Float64,Float64,Float64,Float64,Cat…,Cat…,Float64
1,-0.522784,1,0,0,0,1,0.102805,-0.695947,-0.264208,0.807696,1,1,-0.870976


In [33]:
single_prediction = model(transpose(Matrix(standard_data))) .> 0.5

1×1 BitMatrix:
 0

#### Determine accuracy of the test set. 

In [34]:
test_output = model(transpose(Matrix(X_test)))

1×2000 Matrix{Float32}:
 0.917261  -2.5695  0.143176  -3.15462  …  -1.07569  -1.79366  -4.87876

In [35]:
ConfusionMatrix()(test_output[1, :] .> 0.5, coerce(y_test, OrderedFactor))

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │    1535     │     253     │
├─────────────┼─────────────┼─────────────┤
│      1      │     33      │     179     │
└─────────────┴─────────────┴─────────────┘


In [36]:
mean((test_output[1, :] .> 0.5) .== y_test)

0.857

LoadError: UndefVarError: tst not defined